# 第七章 文本扩展

<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#一引言" data-toc-modified-id="一、引言">一、引言</a></span></li>
        <li>
            <span><a href="#二定制客户邮件" data-toc-modified-id="二、定制客户邮件">二、定制客户邮件</a></span>
        </li>
        <li><span><a href="#三引入温度系数" data-toc-modified-id="三、引入温度系数">三、引入温度系数</a></span>
        </li>
    </ul>
</div>

## 一、引言

扩展是将短文本（例如一组说明或主题列表）输入到大型语言模型中，让模型生成更长的文本（例如基于某个主题的电子邮件或论文）。这种应用是一把双刃剑，好处例如将大型语言模型用作头脑风暴的伙伴；但也存在问题，例如某人可能会使用它来生成大量垃圾邮件。因此，当你使用大型语言模型的这些功能时，请仅以**负责任** (responsible) 和**有益于人们** (helps people) 的方式使用它们。

在本章中，你将学会如何基于 OpenAI API 生成*针对每位客户评价优化*的客服电子邮件。我们还将利用模型的另一个输入参数称为温度，这种参数允许您在模型响应中变化探索的程度和多样性。

同以上几章，你需要类似的代码来配置一个可以使用 OpenAI API 的环境

In [5]:
import openai
# 导入第三方库

import os
openai.api_key = os.environ.get("OPENAI_API_KEY")
# 设置 API_KEY, 请替换成您自己的 API_KEY
deployment_id = None
API_TYPE=os.getenv("OPENAI_API_TYPE", None)
if not API_TYPE is None:
    openai.api_type=API_TYPE
    openai.api_base=os.environ["OPENAI_API_BASE"]
    openai.api_version=os.environ["OPENAI_API_VERSION"]
    deployment_id=os.getenv("DEPLOYMENT_ID", "gpt-35-turbo")

In [6]:
# 一个封装 OpenAI 接口的函数，参数为 Prompt，返回对应结果
def get_completion(prompt, model="gpt-3.5-turbo", temperature=0):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    temperature: 温度系数
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=deployment_id,
        model=model,
        messages=messages,
        temperature=temperature, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]


## 二、定制客户邮件

我们将根据客户评价和情感，针对性写自动回复邮件。因此，我们将给定客户评价和情感，使用 LLM 针对性生成响应，即根据客户评价和评论情感生成定制电子邮件。

我们首先给出一个示例，包括一个评论及对应的情感。

In [7]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
So, they still had the 17 piece system on seasonal \
sale for around $49 in the month of November, about \
half off, but for some reason (call it price gouging) \
around the second week of December the prices all went \
up to about anywhere from between $70-$89 for the same \
system. And the 11 piece system went up around $10 or \
so in price also from the earlier sale price of $29. \
So it looks okay, but if you look at the base, the part \
where the blade locks into place doesn’t look as good \
as in previous editions from a few years ago, but I \
plan to be very gentle with it (example, I crush \
very hard items like beans, ice, rice, etc. in the \ 
blender first then pulverize them in the serving size \
I want in the blender then switch to the whipping \
blade for a finer flour, and use the cross cutting blade \
first when making smoothies, then use the flat blade \
if I need them finer/less pulpy). Special tip when making \
smoothies, finely cut and freeze the fruits and \
vegetables (if using spinach-lightly stew soften the \ 
spinach then freeze until ready for use-and if making \
sorbet, use a small to medium sized food processor) \ 
that you plan to use that way you can avoid adding so \
much ice if at all-when making your smoothie. \
After about a year, the motor was making a funny noise. \
I called customer service but the warranty expired \
already, so I had to buy another one. FYI: The overall \
quality has gone done in these types of products, so \
they are kind of counting on brand recognition and \
consumer loyalty to maintain sales. Got it in about \
two days.
"""

In [8]:
# 我们可以在推理那章学习到如何对一个评论判断其情感倾向
sentiment_zh = "negative"

# 一个产品的评价
review_zh = f"""
他们在11月份的季节性销售期间以约49美元的价格出售17件套装，折扣约为一半。\
但由于某些原因（可能是价格欺诈），到了12月第二周，同样的套装价格全都涨到了70美元到89美元不等。\
11件套装的价格也上涨了大约10美元左右。\
虽然外观看起来还可以，但基座上锁定刀片的部分看起来不如几年前的早期版本那么好。\
不过我打算非常温柔地使用它，例如，\
我会先在搅拌机中将像豆子、冰、米饭等硬物研磨，然后再制成所需的份量，\
切换到打蛋器制作更细的面粉，或者在制作冰沙时先使用交叉切割刀片，然后使用平面刀片制作更细/不粘的效果。\
制作冰沙时，特别提示：\
将水果和蔬菜切碎并冷冻（如果使用菠菜，则轻轻煮软菠菜，然后冷冻直到使用；\
如果制作果酱，则使用小到中号的食品处理器），这样可以避免在制作冰沙时添加太多冰块。\
大约一年后，电机发出奇怪的噪音，我打电话给客服，但保修已经过期了，所以我不得不再买一个。\
总的来说，这些产品的总体质量已经下降，因此它们依靠品牌认可和消费者忠诚度来维持销售。\
货物在两天内到达。
"""

我们已经使用推断课程中所学方法提取了情感，这是一个关于搅拌机的客户评价，现在我们将根据情感定制回复。

以下述 Prompt 为例：假设你是一个客户服务 AI 助手，你的任务是为客户发送电子邮件回复，根据通过三个反引号分隔的客户电子邮件，生成一封回复以感谢客户的评价。

In [9]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt)
print(response)

Dear Valued Customer,

Thank you for taking the time to share your review with us. We appreciate your feedback and apologize for any inconvenience you may have experienced.

We are sorry to hear about the price increase you noticed in December. We strive to provide competitive pricing for our products, and we understand your frustration. If you have any further concerns regarding pricing or any other issues, we encourage you to reach out to our customer service team. They will be more than happy to assist you.

We also appreciate your feedback regarding the base of the system. We continuously work to improve the quality of our products, and your comments will be taken into consideration for future enhancements.

We apologize for any inconvenience caused by the motor issue you encountered. Our customer service team is always available to assist with any warranty-related concerns. We understand that the warranty had expired, but we would still like to address this matter further. Please 

In [10]:
prompt = f"""
你是一位客户服务的AI助手。
你的任务是给一位重要客户发送邮件回复。
根据客户通过“```”分隔的评价，生成回复以感谢客户的评价。提醒模型使用评价中的具体细节
用简明而专业的语气写信。
作为“AI客户代理”签署电子邮件。
客户评论：
```{review_zh}```
评论情感：{sentiment_zh}
"""
response = get_completion(prompt)
print(response)

尊敬的客户，

非常感谢您对我们产品的评价和反馈。我们非常重视您的意见，并对您在使用过程中遇到的问题表示诚挚的歉意。

关于价格上涨的问题，我们理解您的担忧。我们会进一步调查并核实这一情况，以确保我们的价格策略是公正和透明的。

对于产品质量方面的问题，我们深感抱歉。我们一直致力于提供高品质的产品，并且会将您的反馈传达给我们的研发团队，以便改进和提升产品的质量。

同时，我们非常感谢您对我们产品的使用建议和技巧分享。您的经验和建议对我们来说非常宝贵，我们会将其纳入考虑，并在未来的产品设计中加以改进。

如果您有任何其他问题或需要进一步的帮助，请随时与我们联系。我们将竭诚为您提供优质的客户服务。

再次感谢您对我们的支持和反馈。

祝您生活愉快！

AI客户代理


## 三、引入温度系数

接下来，我们将使用语言模型的一个称为“温度” (Temperature) 的参数，它将允许我们改变模型响应的多样性。您可以将温度视为模型探索或随机性的程度。

例如，在一个特定的短语中，“我的最爱食品”最有可能的下一个词是“比萨”，其次最有可能的是“寿司”和“塔可”。因此，在温度为零时，模型将总是选择最有可能的下一个词，而在较高的温度下，它还将选择其中一个不太可能的词，在更高的温度下，它甚至可能选择塔可，而这种可能性仅为五分之一。您可以想象，随着模型继续生成更多单词的最终响应，“我的最爱食品是比萨”将会与第一个响应“我的最爱食品是塔可”产生差异。随着模型的继续，这两个响应也将变得越来越不同。

一般来说，在构建需要可预测响应的应用程序时，我建议**设置温度为零**。在所有课程中，我们一直设置温度为零，如果您正在尝试构建一个可靠和可预测的系统，我认为您应该选择这个温度。如果您尝试以更具创意的方式使用模型，可能需要更广泛地输出不同的结果，那么您可能需要使用更高的温度。

同一段来信，我们提醒模型使用用户来信中的详细信息，并设置温度：

In [11]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_completion(prompt, temperature=0.7)
print(response)

Dear Valued Customer,

Thank you for taking the time to share your feedback with us. We sincerely apologize for any inconvenience you experienced regarding the pricing changes and the quality of our product.

We understand your disappointment with the increase in prices after the seasonal sale. We strive to offer competitive pricing and we apologize for any confusion this may have caused. We appreciate your loyalty and we value your feedback. We will take this into consideration as we continue to improve our pricing strategy.

We also apologize for the issue you encountered with the motor noise and the expired warranty. We understand how frustrating this can be. If you have any further concerns or questions, please do not hesitate to reach out to our dedicated customer service team. They will be more than happy to assist you and find a suitable solution.

We appreciate your support and understanding. Your satisfaction is our utmost priority, and we will continue to work hard to improve

In [12]:
prompt = f"""
你是一名客户服务的AI助手。
你的任务是给一位重要的客户发送邮件回复。
根据通过“```”分隔的客户电子邮件生成回复，以感谢客户的评价。
如果情感是积极的或中性的，感谢他们的评价。
如果情感是消极的，道歉并建议他们联系客户服务。
请确保使用评论中的具体细节。
以简明和专业的语气写信。
以“AI客户代理”的名义签署电子邮件。
客户评价：```{review_zh}```
评论情感：{sentiment_zh}
"""
response = get_completion(prompt, temperature=0.7)
print(response)

尊敬的客户，

非常感谢您对我们产品的评价。我们非常重视您的反馈和意见。

首先，我们对因价格上涨给您带来的不便表示最诚挚的歉意。我们理解您对于产品价格的关注，我们会进一步调查和核实此事，并确保类似情况不再发生。我们会努力提供公正和透明的价格政策。

关于产品质量问题，我们深感抱歉。我们以高品质的产品为宗旨，并且致力于不断改进和提升产品的性能。我们会认真对待您提到的基座锁定刀片的问题，并将其传达给我们的产品团队，以便他们进行改进和优化。

此外，我们感谢您分享了您对于产品的使用方法和建议。您的经验对于我们来说非常宝贵，我们会将其作为改进产品的参考。

最后，非常抱歉您的电机出现了问题，并且保修期已过。我们建议您尽快联系我们的客户服务部门，他们将为您提供更多有关维修和替换的信息，并尽力解决您的问题。

我们非常重视您作为我们重要客户的支持和忠诚。我们将不断努力提供更好的产品和服务，希望能够再次为您提供满意的购物体验。

谢谢您对我们的支持和理解。

祝您生活愉快！

AI客户代理


在温度为零时，每次执行相同的 Prompt ，您获得的回复理应相同。而使用温度为 0.7 时，则每次都会获得不同的输出。

所以，您可以看到它与我们之前收到的电子邮件不同。再次执行将再次获得不同的电子邮件。

因此，我建议您自己尝试温度，以查看输出如何变化。总之，在更高的温度下，模型的输出更加随机。您几乎可以将其视为在更高的温度下，助手**更易分心**，但也许**更有创造力**。